In [1]:
import numpy as np
import seaborn as sns
import os
from matplotlib import pyplot as plt
from ATLAS_data.effFunctions import eventEff,vertexEff
from ATLAS_data.atlasBins import atlas_bins
from helper import LLP
from atlas_susy_2018_13_Recast import getLLPs,getJets,getDisplacedJets,eventAcc,vertexAcc,getModelDict

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

plt.rcParams.update({"savefig.dpi" : 300}) #Figure resolution

delphesDir = os.path.abspath("./DelphesLLP")
os.environ['ROOT_INCLUDE_PATH'] = os.path.join(delphesDir,"external")

import ROOT
import xml.etree.ElementTree as ET


ROOT.gSystem.Load(os.path.join(delphesDir,"libDelphes.so"))

ROOT.gInterpreter.Declare('#include "classes/SortableObject.h"')
ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')


#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'Times New Roman', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
cm = plt.colormaps['YlGnBu']

Welcome to JupyROOT 6.30/06


In [10]:
inputFile = './hs_test/Events/run_01/hs_16_tau_3.00e-03_delphes_events.root'

In [33]:
f = ROOT.TFile(inputFile,'read')
tree = f.Get("Delphes")
nevts = tree.GetEntries()
ntotal = 0
totalweightPB = 0.0
norm = 1.0/nevts
lumi = 139.0
pts = []
for ievt in range(nevts):    
    ntotal += 1
    tree.GetEntry(ievt)   
    weightPB = tree.Event.At(0).Weight
    ns = weightPB*1e3*lumi # number of signal events

    llps = getLLPs(tree.bsm,tree.bsmDirectDaughters,tree.bsmFinalDaughters)    
    # print(len(tree.bsm))
    jets = getJets(tree.GenJet,pTmin=25.,etaMax=5.0)
    # print(f'Nj = {len(jets)}')
    # for j in jets:
        # print(j.PT)

    jetsDisp = getDisplacedJets(jets,llps)
    # print(f'Ndisp = {len(jetsDisp)}')
    # for j in jetsDisp:
        # print(j.PT)

    # Event acceptance:
    highPT_acc = eventAcc(jets,jetsDisp,sr='HighPT')
    trackless_acc = eventAcc(jets,jetsDisp,sr='Trackless')              

    if (highPT_acc+trackless_acc > 0):
        print(ievt,'Acc = ',highPT_acc,trackless_acc)          

    if (not highPT_acc) and (not trackless_acc):
        continue
    
    # Event efficiency
    highPT_eff = eventEff(jets,llps,sr='HighPT')
    trackless_eff = eventEff(jets,llps,sr='Trackless')

    # Vertex acceptances:
    v_acc = np.array([vertexAcc(llp,Rmax=300.0,zmax=300.0,Rmin=4.0,
                                d0min=2.0,nmin=0,mDVmin=0.0)  for llp in llps])
    
    # Vertex efficiencies:
    v_eff = np.array([vertexEff(llp) for llp in llps])

    wvertex = 1.0-np.prod(1.0-v_acc*v_eff)

    print('eff=',ns*highPT_acc*highPT_eff*wvertex,ns*trackless_acc*trackless_eff*wvertex)

f.Close()

762 Acc =  0.0 1.0
[0. 1.]
[False  True]
[[1]]


ValueError: either both or neither of x and y should be given